In [780]:
import time
import pandas as pd
from urllib.request import urlopen
import json
from bs4 import BeautifulSoup

def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)


def get_numbers(i):  
    postno = dataframe2['Post Number'][i]
    replies = dataframe2['Replies'][i]
    return(postno, replies)



df_all = []
i=0
dataframe = pd.DataFrame(columns=['Subject', 'Comment', 'Post Number', 'Replies'])
for i in range(10):
    i=i+1
    url = (("https://a.4cdn.org/biz/") + str(i) + '.json')
    data = get_jsonparsed_data(url)

    list = len(data['threads'])
   
    for i in range(0, list): 
      try:
          comment = data['threads'][i]['posts'][0]['com']
          subject = data['threads'][i]['posts'][0]['sub']
          postno = data['threads'][i]['posts'][0]['no']
          replies = data['threads'][i]['posts'][0]['replies']
      except KeyError:
          subject = 'No subject'
          postno = data['threads'][i]['posts'][0]['no']
          replies = data['threads'][i]['posts'][0]['replies']
      except KeyError:
          comment = data['threads'][i]['posts'][0]['sub']
      dataframe = dataframe.append({'Subject':subject, 'Comment':comment, 'Post Number':postno, 'Replies':replies}, ignore_index=True)
    time.sleep(.02)
    i=i+1

dataframe

,Subject,Comment,Post Number,Replies
0,NO BEGGING,"<span style=""font-weight:600;font-size:150%;li...",4884770,0
1,Welcome to /biz/ - Business &amp; Finance,This board is for the discussion of topics rel...,21374000,1
2,Frens... This year has been hard. Wife has MS,My post from about a year ago.<br>https://waro...,44539830,7
3,No subject,Guess whats next.,44538176,14
4,No subject,"<span class=""quote"">&gt;mfw it&#039;s not over...",44539912,5
...,...,...,...,...
195,Are they really making shitcoins?,During the Wall Street Journal&#039;s CEO Summ...,44524142,44
196,No subject,Doesn&#039;t&#039; he make you hard /biz/?<br>...,44538535,0
197,No subject,What is a good business to start if you have z...,44538148,6
198,No subject,What is a good business to start if you have z...,44538519,1


,Subject,Comment,Post Number,Replies,reply_list,tim_list,ext_list
0,NO BEGGING,"<span style=""font-weight:600;font-size:150%;li...",4884770,0,[],0,0
1,Welcome to /biz/ - Business &amp; Finance,This board is for the discussion of topics rel...,21374000,1,[This board is for the discussion of topics re...,0,0
2,No subject,I AM FINANCIALLY OBLITERATED,44523439,0,[],0,0
3,/GME/ - Did the DD Edition,"<span class=""quote"">&gt;Basic Information</spa...",44521702,121,"[<span class=""quote"">&gt;Basic Information</sp...",0,0
4,No subject,"THEY DIDN&#039;T EVEN SAY ANYTHING, WHY ARE YO...",44523125,5,"[THEY DIDN&#039;T EVEN SAY ANYTHING, WHY ARE Y...",0,0
...,...,...,...,...,...,...,...
195,Oceidon Comfy,Who else is Comfy with Oceidon?,44521238,1,[Who else is Comfy with Oceidon?],0,0
196,No subject,Why hasn&#039;t anyone found a viable way to u...,44521744,0,https://a.4cdn.org/biz/thread/44521744.json,0,0
197,No subject,Literally infinite free money if you are short...,44520938,7,[Literally infinite free money if you are shor...,0,0
198,No subject,henlo? not fucking selling,44521640,4,"[henlo? not fucking selling, checking if im ba...",0,0


In [ ]:
df2 = dataframe.copy
dataframe2['tim_list'] = 0
dataframe2['ext_list'] = 0

for i in range(0,200):

  try:
    postno, replies = get_numbers(i) 
    url = (("https://a.4cdn.org/biz/thread/") + str(postno) + '.json')
    data = get_jsonparsed_data(url)
    replies_text = []
    extensions = []
    images = []
    for j in range(0,replies):
      try:
        reply = data['posts'][j]['com']
        img = data['posts'][j]['tim']
        ext = data['posts'][j]['ext']
        replies_text.append(reply)
        images.append(str(img))
        extensions.append(str(ext))
        dataframe2['reply_list'][i] = replies_text
        dataframe2['tim_list'][i] = images
        dataframe2['ext_list'][i] = extensions
      except KeyError:
        pass
  except HTTPError:
    dataframe2['reply_list'][i] = '404'
    dataframe2['tim_list'][i] = '404'
    dataframe2['ext_list'][i] = '404'
  else:
    pass
time.sleep(1)


In [782]:
df2 = dataframe.copy
dataframe2['tim_list'] = 0
dataframe2['ext_list'] = 0
dataframe2['combined'] = 0
for i in range(0,200):

  try:
    postno, replies = get_numbers(i) 
    url = (("https://a.4cdn.org/biz/thread/") + str(postno) + '.json')
    data = get_jsonparsed_data(url)
    replies_text = []
    extensions = []
    images = []
    combined = []
    for j in range(0,replies):
      try:
        reply = data['posts'][j]['com']
        img = data['posts'][j]['tim']
        ext = data['posts'][j]['ext']
        combined.append(str(img)+str(ext))
        replies_text.append(reply)
        images.append(str(img))
        extensions.append(str(ext))
        dataframe2['reply_list'][i] = replies_text
        dataframe2['tim_list'][i] = images
        dataframe2['ext_list'][i] = extensions
        dataframe2['combined'][i] = combined 
      except KeyError:
        pass
  except HTTPError:
    dataframe2['reply_list'][i] = '404'
    dataframe2['tim_list'][i] = '404'
    dataframe2['ext_list'][i] = '404'
  else:
    pass
time.sleep(1)


C:\Users\Bernie\AppData\Local\Temp/ipykernel_17396/589801797.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe2['reply_list'][i] = replies_text
C:\Users\Bernie\AppData\Local\Temp/ipykernel_17396/589801797.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe2['tim_list'][i] = images
C:\Users\Bernie\AppData\Local\Temp/ipykernel_17396/589801797.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe2['ext_lis

In [785]:
dataframe2

#https://i.4cdn.org/biz/1639581582347.jpg

,Subject,Comment,Post Number,Replies,reply_list,tim_list,ext_list,combined
0,NO BEGGING,"<span style=""font-weight:600;font-size:150%;li...",4884770,0,[],0,0,0
1,Welcome to /biz/ - Business &amp; Finance,This board is for the discussion of topics rel...,21374000,1,[This board is for the discussion of topics re...,[1597354727695],[.png],[1597354727695.png]
2,No subject,I AM FINANCIALLY OBLITERATED,44523439,0,[],0,0,0
3,/GME/ - Did the DD Edition,"<span class=""quote"">&gt;Basic Information</spa...",44521702,121,"[<span class=""quote"">&gt;Basic Information</sp...","[1639581582347, 1639581598138, 1639581726046, ...","[.jpg, .jpg, .jpg, .png, .jpg, .png, .png, .jp...","[1639581582347.jpg, 1639581598138.jpg, 1639581..."
4,No subject,"THEY DIDN&#039;T EVEN SAY ANYTHING, WHY ARE YO...",44523125,5,"[THEY DIDN&#039;T EVEN SAY ANYTHING, WHY ARE Y...",[1639584009550],[.jpg],[1639584009550.jpg]
...,...,...,...,...,...,...,...,...
195,Oceidon Comfy,Who else is Comfy with Oceidon?,44521238,1,[Who else is Comfy with Oceidon?],[1639580692453],[.jpg],[1639580692453.jpg]
196,No subject,Why hasn&#039;t anyone found a viable way to u...,44521744,0,https://a.4cdn.org/biz/thread/44521744.json,0,0,0
197,No subject,Literally infinite free money if you are short...,44520938,7,[Literally infinite free money if you are shor...,"[1639580020864, 1639580147234, 1639580357185, ...","[.png, .png, .jpg, .jpg]","[1639580020864.png, 1639580147234.png, 1639580..."
198,No subject,henlo? not fucking selling,44521640,4,"[henlo? not fucking selling, <a href=""#p445216...","[1639581484181, 1639581579402]","[.jpg, .png]","[1639581484181.jpg, 1639581579402.png]"


0                                                      0
1                                        [1597354727695]
2                                                      0
3      [1639581582347, 1639581598138, 1639581726046, ...
4                                        [1639584009550]
                             ...                        
195                                      [1639580692453]
196                                                    0
197    [1639580020864, 1639580147234, 1639580357185, ...
198                       [1639581484181, 1639581579402]
199                                      [1639581413487]
Name: tim_list, Length: 200, dtype: object

In [ ]:
#CATALOG

url = ("https://a.4cdn.org/biz/5.json")
data = get_jsonparsed_data(url)

In [ ]:

num_replies = int(data['posts'][0]['replies'])
replies = []

for i in range(num_replies):
    reply = data['posts'][i]['com']
    replies.append(reply)

replies

soup = BeautifulSoup() 
def beautify (reply): 
  soup = BeautifulSoup(reply) 
  reply = soup.get_text() 
  return reply

#df["clean_text"] = df.description.apply(lambda text: beautify(reply))

replies


In [ ]:
''''This is to get the replies into a long string for NLP, need to use regex too

#stringy = str(replies)
#soup = BeautifulSoup(stringy)
#print(soup.get_text())

'''



In [178]:
thread = 44524361
data = get_jsonparsed_data("https://a.4cdn.org/biz/thread/" + str(thread) + '.json')

data['posts'][0]

## https://i.4cdn.org/[board]/[4chan image ID].[file extension]
## 'tim' is the name of the jpg/png
## 'ext' is the extension

1639586017436

In [790]:
import os
import shutil
import requests
import urllib.request
import urllib
import requests
combined = dataframe2['combined']

In [801]:
#DOWNLOAD ALL IMAGES FOR THREAD/IN THREADS

links = []
newlinks = []
combined = dataframe2['combined']

for i in range(0, len(combined)):
    if combined[i] == 0:
        links.append('No image')
    elif combined[i] != 0:
        links = combined[i]
        for i in links:
            newlinks.append(i)
            imgURL = ("https://i.4cdn.org/biz/") + str(i)
            name = str(i)
            print(imgURL)
            r = requests.get(imgURL)
            with open('imgs/'+name, 'wb') as f:
                f.write(r.content)
            time.sleep(.2)
print(name)

0
https://i.4cdn.org/biz/1597354727695.png
https://i.4cdn.org/biz/No image
https://i.4cdn.org/biz/No image
https://i.4cdn.org/biz/No image
https://i.4cdn.org/biz/No image
https://i.4cdn.org/biz/No image
0
https://i.4cdn.org/biz/1639581582347.jpg
https://i.4cdn.org/biz/1639581598138.jpg
https://i.4cdn.org/biz/1639581726046.jpg
https://i.4cdn.org/biz/1639581730129.png
https://i.4cdn.org/biz/1639581750652.jpg
https://i.4cdn.org/biz/1639581775739.png
https://i.4cdn.org/biz/1639581780663.png
https://i.4cdn.org/biz/1639581839067.jpg
https://i.4cdn.org/biz/1639581851027.gif
https://i.4cdn.org/biz/1639581894245.jpg
https://i.4cdn.org/biz/1639581916960.jpg
https://i.4cdn.org/biz/1639582125109.jpg
https://i.4cdn.org/biz/1639582276121.jpg
https://i.4cdn.org/biz/1639582308163.jpg
https://i.4cdn.org/biz/1639582423968.png
https://i.4cdn.org/biz/1639582470274.png
https://i.4cdn.org/biz/1639582719732.jpg
https://i.4cdn.org/biz/1639582790668.jpg
https://i.4cdn.org/biz/1639582822589.jpg
https://i.4cdn.o

In [699]:
links = []
combined = dataframe2['combined']

               if r.status_code == 200:
                    imgURL = ("https://i.4cdn.org/") + str(combined[i][j])
                    name = combined[i][j]
                    r = requests.get(imgURL)
                    print('hi')
                    links.append('imgURL')
                    with open("imgs/{i}.png", 'wb') as f:
                        f.write(r.content)
        else: 
            print("no")




1

In [611]:
dataframe2['combined'].apply(lambda x: str())

dataframe2['conbined']

KeyError: 'conbined'

In [598]:
filenames = combined

urls = []

for i in range(3,4):
    print(filenames[i])
    print('loop #' + str(i))
    try:
        length = len(filenames[i])
        if length > 1:
            for j in length[j]:
                url = ("http://www.4chan.com/" + length[i][1])
    except:
        pass
    print("the length of the list is:" + str(length))
    print(url)

['1639581582347.jpg', '1639581598138.jpg', '1639581726046.jpg', '1639581730129.png', '1639581750652.jpg', '1639581775739.png', '1639581780663.png', '1639581839067.jpg', '1639581851027.gif', '1639581894245.jpg', '1639581916960.jpg', '1639582125109.jpg', '1639582276121.jpg', '1639582308163.jpg', '1639582423968.png', '1639582470274.png', '1639582719732.jpg', '1639582790668.jpg', '1639582822589.jpg', '1639582847207.png', '1639582909407.jpg', '1639582923512.png', '1639582928967.jpg', '1639583598514.jpg', '1639583625556.png', '1639583631513.jpg', '1639583653185.jpg', '1639583671186.gif', '1639583698444.png', '1639583750472.gif', '1639583776815.gif', '1639583792258.jpg', '1639583850859.png', '1639583854001.jpg', '1639583878339.jpg', '1639583984969.jpg', '1639584127879.png', '1639584140266.png', '1639584181259.png', '1639584248999.png', '1639584252786.png', '1639584275913.jpg', '1639584348355.png', '1639584439402.jpg', '1639584473993.jpg', '1639584491326.png']
loop #3
the length of the list is

In [467]:
len(combined[1])


1